En el ámbito de la ciencia de datos y el aprendizaje automático, trabajar con conjuntos de datos desequilibrados es un reto habitual, sobre todo en tareas de clasificación binaria. Los datos desequilibrados se refieren a conjuntos de datos en los que la distribución de las clases está sesgada, **con una clase significativamente superior a las demás**. Esta situación es frecuente en diversos ámbitos, como la detección de fraudes, el diagnóstico médico y la detección de anomalías.

Manejar eficazmente los datos desequilibrados es crucial, ya que los algoritmos tradicionales de aprendizaje automático tienden a funcionar mal en este tipo de conjuntos de datos, favoreciendo a la clase mayoritaria y descuidando a la minoritaria. Aprenderemos sobre las dos clases de técnicas para manejar datos desequilibrados utilizando la biblioteca Imbalance-Learn en Python, junto con árboles de decisión y estrategias de validación cruzada para mejorar la robustez y generalización del modelo.

## Imbalanced-learn

[Imbalanced-Learn](https://imbalanced-learn.org), junto con scikit-learn (sklearn), es una biblioteca de Python diseñada específicamente para abordar el desequilibrio de clases en tareas de aprendizaje automático. Proporciona un conjunto completo de técnicas de remuestreo, enfoques algorítmicos y métodos híbridos para gestionar eficazmente los conjuntos de datos desequilibrados.

Con Imbalance-Learn, los científicos de datos pueden ajustar parámetros para reequilibrar conjuntos de datos, aplicar enfoques algorítmicos para mitigar el sobreajuste, entrenar modelos con datos equilibrados y evaluar el rendimiento del modelo utilizando métricas adecuadas. Tanto para principiantes como para profesionales experimentados, Imbalance-Learn ofrece un útil tutorial que le guiará a la hora de abordar el desequilibrio de clases en sus proyectos de aprendizaje automático.

### Técnicas habituales

* Métodos de remuestreo:

    Las técnicas de sobremuestreo como SMOTE (Synthetic Minority Over-sampling Technique) y ADASYN (Adaptive Synthetic Sampling) generan muestras sintéticas para la clase minoritaria con el fin de equilibrar el conjunto de datos.
    Las técnicas de submuestreo eliminan aleatoriamente muestras de la clase mayoritaria para lograr el equilibrio de clases.

* Enfoques algorítmicos:
 
    El aprendizaje sensible a los costes ajusta los costes de clasificación errónea para tener en cuenta el desequilibrio de clases durante el entrenamiento del modelo.
    Los métodos de ensamblaje, como Random Forest y XGBoost, gestionan de forma inherente el desequilibrio de clases agregando las predicciones de varios modelos. Podemos a su vez favorecer el peso de una de las clases para favorecer su escasez de muestras.

* Métodos híbridos:
    
    Los métodos híbridos combinan técnicas de sobremuestreo y submuestreo para conseguir un conjunto de datos equilibrado, preservando al mismo tiempo la información de los datos originales.


Tomaremos como ejemplo el conjunto de datos de AID: https://www.kaggle.com/datasets/uciml/bioassay-datasets/data

In [28]:
import pandas as pd

data = pd.read_csv('data/AID1284Morered_test.csv')
data.head()

,ARC_01_ARC,ARC_02_ARC,ARC_03_ARC,ARC_04_ARC,ARC_05_ARC,ARC_06_ARC,ARC_07_ARC,ARC_01_POS,ARC_02_POS,ARC_03_POS,...,WBN_LP_H_1.00,XLogP,PSA,NumRot,NumHBA,NumHBD,MW,BBB,BadGroup,Outcome
0,0,0,0,0,0,0,0,0,0,0,...,3.60912,3.363,87.74,9,7,2,409.486,0,0,Active
1,0,0,1,0,0,0,0,0,0,0,...,3.73406,1.770,66.76,6,5,0,382.210,1,1,Active
2,1,0,0,0,1,1,0,0,0,0,...,3.84241,2.964,103.79,9,6,2,382.416,0,0,Active
3,0,0,0,0,1,0,0,0,0,0,...,3.50705,0.470,128.12,8,7,2,350.396,0,0,Active
4,1,0,0,0,0,1,0,0,0,0,...,3.95624,0.357,188.86,8,5,2,434.568,0,1,Active


In [29]:
data['Outcome'].value_counts(normalize=True)

Outcome
Inactive    0.847222
Active      0.152778
Name: proportion, dtype: float64

In [30]:
data.dtypes

ARC_01_ARC      int64
ARC_02_ARC      int64
ARC_03_ARC      int64
ARC_04_ARC      int64
ARC_05_ARC      int64
               ...   
NumHBD          int64
MW            float64
BBB             int64
BadGroup        int64
Outcome        object
Length: 915, dtype: object

In [31]:
X = data.drop(columns="Outcome")
y = (data["Outcome"] == "Active").astype(int)

In [32]:
# !pip install xgboost

In [33]:
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators=3)
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [34]:
from sklearn.metrics import classification_report

print(classification_report(y, model.predict(X), zero_division=0))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92        61
           1       0.00      0.00      0.00        11

    accuracy                           0.85        72
   macro avg       0.42      0.50      0.46        72
weighted avg       0.72      0.85      0.78        72



In [35]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y, model.predict(X))

0.5

#### Enfoque algorítmico

Probaremos a asignar un mayor peso a la clase desfavorecida.

In [36]:
data['Outcome'].unique()

array(['Active', 'Inactive'], dtype=object)

In [37]:
import numpy as np
from sklearn.utils import class_weight

classes_weights = list(class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(data['Outcome']), y=data['Outcome']))
classes_weights

[3.272727272727273, 0.5901639344262295]

In [38]:
weights = np.ones(X.shape[0], dtype = 'float')
for i, val in enumerate(y):
    weights[i] = classes_weights[val-1]

In [39]:
model = xgb.XGBClassifier(n_estimators=3)
model.fit(X, y, sample_weight=weights)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [40]:
print(classification_report(y, model.predict(X), zero_division=0))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        61
           1       0.85      1.00      0.92        11

    accuracy                           0.97        72
   macro avg       0.92      0.98      0.95        72
weighted avg       0.98      0.97      0.97        72



In [41]:
roc_auc_score(y, model.predict(X))

0.9836065573770492

Los modelos basados en árboles en su modalidad boosting tienen capacidad de ajuste muy fino si se les permite iterar los suficiente (n_estimators)

In [42]:
model = xgb.XGBClassifier(n_estimators=100)
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [43]:
print(classification_report(y, model.predict(X), zero_division=0))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        61
           1       1.00      1.00      1.00        11

    accuracy                           1.00        72
   macro avg       1.00      1.00      1.00        72
weighted avg       1.00      1.00      1.00        72



In [44]:
roc_auc_score(y, model.predict(X))

1.0

Veamos cómo resultaría aplicándolo contra una separación de training y test.

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X, y, weights, test_size=0.2, random_state=42)

In [46]:
model = xgb.XGBClassifier(n_estimators=100)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [47]:
roc_auc_score(y_test, model.predict(X_test))

0.45

In [48]:
print(classification_report(y_test, model.predict(X_test), zero_division=0))

              precision    recall  f1-score   support

           0       0.64      0.90      0.75        10
           1       0.00      0.00      0.00         5

    accuracy                           0.60        15
   macro avg       0.32      0.45      0.38        15
weighted avg       0.43      0.60      0.50        15



In [49]:
model = xgb.XGBClassifier(n_estimators=20)
model.fit(X_train, y_train, sample_weight=w_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [50]:
roc_auc_score(y_test, model.predict(X_test))

0.3

In [24]:
print(classification_report(y_test, model.predict(X_test), zero_division=0))

              precision    recall  f1-score   support

           0       0.55      0.60      0.57        10
           1       0.00      0.00      0.00         5

    accuracy                           0.40        15
   macro avg       0.27      0.30      0.29        15
weighted avg       0.36      0.40      0.38        15



### Under sampling

Las técnicas de submuestreo ayudan a equilibrar la distribución de clases para una distribución de clases sesgada. Una distribución de clases desequilibrada tiene más ejemplos de una o más clases (clase mayoritaria) y pocos ejemplos pertenecientes a clases minoritarias.

Las técnicas de submuestreo eliminan algunos ejemplos del conjunto de datos de entrenamiento pertenecientes a la clase mayoritaria. Se trata de equilibrar mejor la distribución de clases reduciendo la asimetría de 1:80 a 1:5 o 1:1. El submuestreo se utiliza junto con un método de sobremuestreo. La combinación de estas técnicas suele dar mejores resultados que el uso de cualquiera de ellas por separado.

La técnica básica de submuestreo elimina los ejemplos aleatoriamente de la clase mayoritaria, lo que se denomina "submuestreo aleatorio". Aunque esta técnica es sencilla y a veces también eficaz, existe el riesgo de perder información útil o importante que podría determinar el límite de decisión entre las clases.

Por lo tanto, es necesario un enfoque más heurístico que pueda elegir ejemplos para no borrar y ejemplos redundantes para borrar. Afortunadamente, algunas técnicas de submuestreo utilizan este tipo de heurística.

In [53]:
!pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: C:\Program Files\Python310\python.exe -m pip install --upgrade pip


In [52]:
from collections import Counter
from imblearn.under_sampling import NearMiss

undersampler = NearMiss()
X_resampled, y_resampled = undersampler.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
model = xgb.XGBClassifier(n_estimators=100)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
print(classification_report(y_test, model.predict(X_test), zero_division=0))

              precision    recall  f1-score   support

           0       0.64      0.90      0.75        10
           1       0.00      0.00      0.00         5

    accuracy                           0.60        15
   macro avg       0.32      0.45      0.38        15
weighted avg       0.43      0.60      0.50        15



In [ ]:
roc_auc_score(y_test, model.predict(X_test))

0.45

In [ ]:
model.fit(X_resampled, y_resampled)
print(classification_report(y_test, model.predict(X_test), zero_division=0))

NameError: name 'X_resampled' is not defined

In [ ]:
roc_auc_score(y_test, model.predict(X_test))

0.45

#### Condensed nearest Neighbors

https://imbalanced-learn.org/stable/under_sampling.html#condensed-nearest-neighbors

In [ ]:
from imblearn.under_sampling import CondensedNearestNeighbour

undersampler = CondensedNearestNeighbour()
X_resampled, y_resampled = undersampler.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
model.fit(X_resampled, y_resampled)
print(classification_report(y_test, model.predict(X_test), zero_division=0))

NameError: name 'X_resampled' is not defined

In [ ]:
roc_auc_score(y_test, model.predict(X_test))

0.45

### Over sampling

A diferencia del submuestreo, que se centra en eliminar los ejemplos de la clase mayoritaria, el sobremuestreo se centra en aumentar las muestras de la clase minoritaria.

También podemos duplicar los ejemplos para aumentar las muestras de clase minoritaria. Aunque equilibra los datos, no proporciona información adicional al modelo de clasificación.

Por lo tanto, es necesario sintetizar nuevos ejemplos utilizando una técnica adecuada. Aquí entran en escena SMOTE y ADASYN.

#### Synthetic Minority Oversampling Technique 

[SMOTE](https://arxiv.org/abs/1106.1813) genera muestras sintéticas de la clase minoritaria, aumentando así el número de muestras de la clase minoritaria. Para ello, selecciona una muestra de la clase minoritaria, busca sus vecinos más cercanos y crea nuevas muestras interpolando entre la muestra seleccionada y sus vecinos más cercanos. Este proceso ayuda a equilibrar el conjunto de datos, lo que permite a los modelos de aprendizaje automático aprender más eficazmente de la clase minoritaria y mejorar su rendimiento general en conjuntos de datos desequilibrados.

#### Adaptive Synthetic Sampling

[ADASYN](https://ieeexplore.ieee.org/document/4633969) adapta el número de muestras sintéticas generadas en función de la dificultad de aprendizaje de cada instancia de clase minoritaria. ADASYN selecciona una instancia de clase minoritaria, encuentra sus vecinos más cercanos y, a continuación, genera una muestra sintética interpolando entre la instancia seleccionada y sus vecinos más cercanos. El número de muestras sintéticas generadas es proporcional a la dificultad de aprendizaje de la instancia, generándose más muestras para las instancias más difíciles de aprender. Esto ayuda a centrar la atención del modelo en las instancias más difíciles, mejorando el rendimiento de clasificación de los modelos de aprendizaje automático en conjuntos de datos desequilibrados.

In [36]:
from imblearn.over_sampling import SMOTE, ADASYN

X_resampled, y_resampled = SMOTE().fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

ModuleNotFoundError: No module named 'imblearn'

In [37]:
model.fit(X_resampled, y_resampled)
print(classification_report(y_test, model.predict(X_test), zero_division=0))

NameError: name 'X_resampled' is not defined

In [38]:
roc_auc_score(y_test, model.predict(X_test))

0.45

In [39]:
X_resampled, y_resampled = ADASYN().fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

NameError: name 'ADASYN' is not defined

In [40]:
model.fit(X_resampled, y_resampled)
print(classification_report(y_test, model.predict(X_test), zero_division=0))

NameError: name 'X_resampled' is not defined

In [41]:
roc_auc_score(y_test, model.predict(X_test))

0.45